# Neural Style Transfer

*  Jafet Israel sierra Lagos
*  jafet.sierra.l@gmail.com

In [ ]:
#Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image
import matplotlib.pyplot as plt

In [ ]:
model = models.vgg19(pretrained=True).features

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
from torch.serialization import load
class VGG(nn.Module):
  def __init__(self):
    super(VGG,self).__init__()
    self.chosen_features = ['0','5','10','19','28']
    self.model = models.vgg19(pretrained=True).features[:29]
  def forward(self,x):
    features = []
    for num, layer in enumerate(self.model):
      x = layer(x)
      if str(num) in self.chosen_features:
        features.append(x)
    return features

image_size = 720

loader = transforms.Compose([
                             transforms.Resize((image_size,image_size)),
                             transforms.ToTensor(),
])

def load_image(image_name):
  image = Image.open(image_name)
  image = loader(image).unsqueeze(0)
  return image.to(device)

device = torch.device("cuda" if torch.cuda.is_available else "cpu")

In [ ]:
content_img = load_image('profile2.png')
style_img   = load_image('Starry_Night.jpg')

generated   = content_img.clone().requires_grad_(True)

#Model
model = VGG().to(device).eval()


#hyperparameters
epochs = 10001
lr     = 1e-3
alpha  = 0.5
beta   = 0.1

optimizer = optim.Adam([generated],lr=lr)

In [ ]:
#Training
for epoch in range(epochs):
  generated_f = model(generated)
  content_f   = model(content_img)
  style_f     = model(style_img)

  style_loss = 0
  content_loss = 0

  for gen_f, cont_f, st_f in zip(generated_f,content_f,style_f):
    batch_size, channel, height, width = gen_f.shape

    content_loss += torch.mean((gen_f - cont_f)**2)

    #Gram matrix
    #generated gm
    G = gen_f.view(channel,height*width).mm(gen_f.view(channel,height*width).t())
    #style gm
    S = st_f.view(channel,height*width).mm(st_f.view(channel,height*width).t())
    
    style_loss += torch.mean((G-S)**2)
  
  total_loss  = alpha * content_loss + beta * style_loss

  optimizer.zero_grad()
  total_loss.backward()
  optimizer.step()

  if epoch % 1000==0:
    print(f'Epoch: {epoch} - loss: {total_loss}')
    save_image(generated,f'generated_{epoch/1000}.png')

Epoch: 0 - loss: 67395168.0
Epoch: 1000 - loss: 136289.515625
Epoch: 2000 - loss: 61382.08203125
Epoch: 3000 - loss: 36800.77734375
Epoch: 4000 - loss: 22732.7265625
Epoch: 5000 - loss: 14721.65234375
Epoch: 6000 - loss: 10117.4765625
Epoch: 7000 - loss: 7392.0048828125
Epoch: 8000 - loss: 5659.48486328125
Epoch: 9000 - loss: 4538.7958984375
Epoch: 10000 - loss: 3767.140869140625


In [ ]:
generated

tensor([[[[ 0.0957,  0.4492,  0.3655,  ...,  0.1953,  0.5909,  0.6249],
          [ 0.6098,  0.7472,  0.5659,  ...,  0.3508,  0.3554,  0.4762],
          [ 0.7125,  0.6386,  0.7035,  ...,  0.1197,  0.2789,  0.5420],
          ...,
          [ 0.5948,  0.5599,  0.3417,  ..., -0.0574,  0.0577,  0.4617],
          [ 0.5819,  0.5223,  0.2855,  ...,  0.2428,  0.3896,  0.6125],
          [ 0.6395,  0.3739,  0.3024,  ...,  0.2667,  0.5245,  0.6260]],

         [[ 0.3235,  0.3907,  0.5074,  ...,  0.2877,  0.3103,  0.3295],
          [ 0.6588,  0.7333,  0.7376,  ...,  0.3418,  0.4705,  0.9133],
          [ 0.6632,  0.7601,  0.6918,  ...,  0.1647,  0.1881,  0.6574],
          ...,
          [ 0.5342,  0.5763,  0.4082,  ...,  0.0993,  0.1724,  0.6110],
          [ 0.5303,  0.4385,  0.3602,  ...,  0.1920,  0.2635,  0.7188],
          [ 0.5788,  0.5275,  0.3661,  ...,  0.3102,  0.4731,  0.7227]],

         [[ 1.0793,  0.5123,  0.6882,  ...,  0.3944,  0.6315,  0.9132],
          [ 0.7296,  0.5828,  